## Second Highest Salary
[1] https://leetcode.com/problems/second-highest-salary/

Write a SQL query to get the second highest salary from the Employee table.

```
+----+--------+
| Id | Salary |
+----+--------+
| 1  | 100    |
| 2  | 200    |
| 3  | 300    |
+----+--------+
```
For example, given the above Employee table, the query should return 200 as the second highest salary. If there is no second highest salary, then the query should return null.
```
+---------------------+
| SecondHighestSalary |
+---------------------+
| 200                 |
+---------------------+
```
-- **Solution using sub query**
```
SELECT
    MAX(salary) AS SecondHighestSalary
FROM Employee
WHERE salary NOT IN (
    SELECT
        MAX(salary)
    FROM Employee
)
```

--- ***Solution using IFNULL and Offset***
```
SELECT
    IFNULL(
      (SELECT DISTINCT Salary
       FROM Employee
       ORDER BY Salary DESC
        LIMIT 1 OFFSET 1),
    NULL) AS SecondHighestSalary
 ```
 
***------------------------------------------------------------------------------------------------------------------------------------------------------------------------***

## Rank Scores
[2] https://leetcode.com/problems/rank-scores/

Write a SQL query to rank scores. If there is a tie between two scores, both should have the same ranking. Note that after a tie, the next ranking number should be the next consecutive integer value. In other words, there should be no "holes" between ranks.

```
+----+-------+
| Id | Score |
+----+-------+
| 1  | 3.50  |
| 2  | 3.65  |
| 3  | 4.00  |
| 4  | 3.85  |
| 5  | 4.00  |
| 6  | 3.65  |
+----+-------+
```

For example, given the above Scores table, your query should generate the following report (order by highest score):

```
+-------+---------+
| score | Rank    |
+-------+---------+
| 4.00  | 1       |
| 4.00  | 1       |
| 3.85  | 2       |
| 3.65  | 3       |
| 3.65  | 3       |
| 3.50  | 4       |
+-------+---------+
```
Important Note: For MySQL solutions, to escape reserved words used as column names, you can use an apostrophe before and after the keyword. For example `Rank`.

***--- SOLUTION --***

```
SELECT
    score,
    DENSE_RANK() OVER(ORDER BY score DESC) AS `Rank`
FROM Scores
```

***------------------------------------------------------------------------------------------------------------------------------------------------------------------------***

## Customers Who Never Order
[3] https://leetcode.com/problems/customers-who-never-order/

Suppose that a website contains two tables, the Customers table and the Orders table. Write a SQL query to find all customers who never order anything.

Table: Customers.
```
+----+-------+
| Id | Name  |
+----+-------+
| 1  | Joe   |
| 2  | Henry |
| 3  | Sam   |
| 4  | Max   |
+----+-------+
Table: Orders.

+----+------------+
| Id | CustomerId |
+----+------------+
| 1  | 3          |
| 2  | 1          |
+----+------------+
Using the above tables as example, return the following:

+-----------+
| Customers |
+-----------+
| Henry     |
| Max       |
+-----------+
```

**--SOLUTION--** DRAWBACK does handle customers with same Names

``
SELECT DISTINCT
    c.Name AS Customers
FROM Customers c
LEFT JOIN Orders o
ON o.CustomerId = c.Id
WHERE o.CustomerId IS NULL
``

**--using subquery--** 

```
SELECT 
    Name AS Customers
FROM Customers 
WHERE Id NOT IN (
    SELECT
        DISTINCT CustomerId
    FROM Orders
)
```

***------------------------------------------------------------------------------------------------------------------------------------------------------------------------***

## Delete duplicate emails
[4] https://leetcode.com/problems/delete-duplicate-emails/

Write a SQL query to delete all duplicate email entries in a table named Person, keeping only unique emails based on its smallest Id.
```
+----+------------------+
| Id | Email            |
+----+------------------+
| 1  | john@example.com |
| 2  | bob@example.com  |
| 3  | john@example.com |
+----+------------------+
```
Id is the primary key column for this table.
For example, after running your query, the above Person table should have the following rows:
```
+----+------------------+
| Id | Email            |
+----+------------------+
| 1  | john@example.com |
| 2  | bob@example.com  |
+----+------------------+
```
Note:

Your output is the whole Person table after executing your sql. Use delete statement.

** -- SOLUTIION -- **

```
DELETE FROM Person
WHERE Id IN (
    SELECT 
        Id
    FROM (
        SELECT
            Id,
            Email,
            ROW_NUMBER() OVER(PARTITION BY Email ORDER BY id) AS row_num
        FROM Person
    ) AS row_person
    WHERE row_num >1
)
```
***------------------------------------------------------------------------------------------------------------------------------------------------------------------------***

## Rising Temperature
[5] https://leetcode.com/problems/rising-temperature/

Table: Weather

```
+---------------+---------+
| Column Name   | Type    |
+---------------+---------+
| id            | int     |
| recordDate    | date    |
| temperature   | int     |
+---------------+---------+
id is the primary key for this table.
This table contains information about the temperature in a certain day.
 

Write an SQL query to find all dates' id with higher temperature compared to its previous dates (yesterday).

Return the result table in any order.

The query result format is in the following example:

Weather
+----+------------+-------------+
| id | recordDate | Temperature |
+----+------------+-------------+
| 1  | 2015-01-01 | 10          |
| 2  | 2015-01-02 | 25          |
| 3  | 2015-01-03 | 20          |
| 4  | 2015-01-04 | 30          |
+----+------------+-------------+

Result table:
+----+
| id |
+----+
| 2  |
| 4  |
+----+
In 2015-01-02, temperature was higher than the previous day (10 -> 25).
In 2015-01-04, temperature was higher than the previous day (20 -> 30).

```

**-- SOLUTION Handling edge case where a date is missing --**
```
WITH prev_day_temp_details AS (
    SELECT
        id,
        recordDate,
        Temperature,
        LAG(Temperature,1 ) OVER(ORDER BY recordDate) AS prev_temp,
        LAG(recordDate,1 ) OVER(ORDER BY recordDate) AS prev_day
    FROM Weather
)
SELECT
    id
FROM prev_day_temp_details
WHERE 
    Temperature > prev_temp
    AND DATEDIFF(recordDate, prev_day) =1
```

**-- SOLUTION using SELF JOIN and INTERVAL 1 DAY --**
```
WITH stg AS (
    SELECT
        c.id,
        c.recordDate,
        c.Temperature AS current_temp,
        p.recordDate AS prev_date,
        p.Temperature AS pre_temp
    FROM Weather c
    LEFT JOIN Weather p
    ON c.recordDate = (p.recordDate + INTERVAL '1' DAY)
    )
SELECT
    id
FROM stg
WHERE current_temp > pre_temp
```

***------------------------------------------------------------------------------------------------------------------------------------------------------------------------***

## Count Student Number in Departments

[6] https://leetcode.com/problems/count-student-number-in-departments/

A university uses 2 data tables, student and department, to store data about its students and the departments associated with each major.

Write a query to print the respective department name and number of students majoring in each department for all departments in the department table (even ones with no current students).

Sort your results by descending number of students; if two or more departments have the same number of students, then sort those departments alphabetically by department name.

The student is described as follow:
```
| Column Name  | Type      |
|--------------|-----------|
| student_id   | Integer   |
| student_name | String    |
| gender       | Character |
| dept_id      | Integer   |
where student_id is the student's ID number, student_name is the student's name, gender is their gender, and dept_id is the department ID associated with their declared major.

And the department table is described as below:

| Column Name | Type    |
|-------------|---------|
| dept_id     | Integer |
| dept_name   | String  |
where dept_id is the department's ID number and dept_name is the department name.

Here is an example input:
student table:

| student_id | student_name | gender | dept_id |
|------------|--------------|--------|---------|
| 1          | Jack         | M      | 1       |
| 2          | Jane         | F      | 1       |
| 3          | Mark         | M      | 2       |
department table:

| dept_id | dept_name   |
|---------|-------------|
| 1       | Engineering |
| 2       | Science     |
| 3       | Law         |
The Output should be:

| dept_name   | student_number |
|-------------|----------------|
| Engineering | 2              |
| Science     | 1              |
| Law         | 0              |

```

**-- SOLUTION --** 

**One doesn't need to add COALESCE as COUNT(NULL) = 0** 

**should not use COUNT(1) it takes a row counts and not the students**

```
SELECT
    dept_name,
    COUNT(student_id) AS student_number
FROM department
LEFT JOIN student
ON student.dept_id = department.dept_id
GROUP BY
    dept_name
ORDER BY 
    student_number DESC,
    dept_name
```
***------------------------------------------------------------------------------------------------------------------------------------------------------------------------***

## Friend Requests I: Overall Acceptance Rate
[7] https://leetcode.com/problems/friend-requests-i-overall-acceptance-rate/

```
Table: FriendRequest

+----------------+---------+
| Column Name    | Type    |
+----------------+---------+
| sender_id      | int     |
| send_to_id     | int     |
| request_date   | date    |
+----------------+---------+
There is no primary key for this table, it may contain duplicates.
This table contains the ID of the user who sent the request, the ID of the user who received the request, and the date of the request.
 

Table: RequestAccepted

+----------------+---------+
| Column Name    | Type    |
+----------------+---------+
| requester_id   | int     |
| accepter_id    | int     |
| accept_date    | date    |
+----------------+---------+
There is no primary key for this table, it may contain duplicates.
This table contains the ID of the user who sent the request, the ID of the user who received the request, and the date when the request was accepted.

```


**-- SOLUTION-- PAY ATTENTION**

```
SELECT 
    COALESCE((ROUND(accepts/requests, 2)), 0.0) AS accept_rate
FROM
    (
        SELECT 
            COUNT(DISTINCT sender_id, send_to_id) AS requests 
        FROM FriendRequest
    ) AS t1
    JOIN 
    (
        SELECT 
            COUNT(DISTINCT requester_id, accepter_id) AS accepts 
        FROM RequestAccepted
    ) AS t2
```

**-- SOLUTION-- WITHOUT JOIN**
```
WITH requests AS (
SELECT
    DISTINCT 
        sender_id,
        send_to_id
FROM FriendRequest
),
accepts AS (
SELECT
    DISTINCT
    requester_id,
    accepter_id
FROM RequestAccepted
)
SELECT
    COALESCE(
        ROUND( 
         (SELECT COUNT(1) FROM accepts) /
         (SELECT COUNT(1) FROM requests) 
         ,2)
        ,0.0) AS accept_rate
```

***------------------------------------------------------------------------------------------------------------------------------------------------------------------------***

## Friend Requests II: Who Has the Most Friends

[8] https://leetcode.com/problems/friend-requests-ii-who-has-the-most-friends/
    
In social network like Facebook or Twitter, people send friend requests and accept others' requests as well.

```
Table request_accepted

+--------------+-------------+------------+
| requester_id | accepter_id | accept_date|
|--------------|-------------|------------|
| 1            | 2           | 2016_06-03 |
| 1            | 3           | 2016-06-08 |
| 2            | 3           | 2016-06-08 |
| 3            | 4           | 2016-06-09 |
+--------------+-------------+------------+
This table holds the data of friend acceptance, while requester_id and accepter_id both are the id of a person.
 

Write a query to find the the people who has most friends and the most friends number under the following rules:

It is guaranteed there is only 1 people having the most friends.
The friend request could only been accepted once, which mean there is no multiple records with the same requester_id and accepter_id value.
For the sample data above, the result is:

Result table:
+------+------+
| id   | num  |
|------|------|
| 3    | 3    |
+------+------+
The person with id '3' is a friend of people '1', '2' and '4', so he has 3 friends in total, which is the most number than any others.
Follow-up:
In the real world, multiple people could have the same most number of friends, can you find all these people in this case?

```
**-- SOLUTION --**

```
SELECT 
    id,
     COUNT(frd_req)  AS num
FROM (
    SELECT
        requester_id AS id,
        accepter_id AS frd_req
    FROM request_accepted

    UNION ALL

    SELECT
        accepter_id AS id,
        requester_id AS frd_req
    FROM request_accepted
) AS requests
GROUP BY 
    id
ORDER BY
    num DESC
LIMIT 1
```
***------------------------------------------------------------------------------------------------------------------------------------------------------------------------***

## Game Play Analysis III

[9] https://leetcode.com/problems/game-play-analysis-iii/

Table: Activity

```
+--------------+---------+
| Column Name  | Type    |
+--------------+---------+
| player_id    | int     |
| device_id    | int     |
| event_date   | date    |
| games_played | int     |
+--------------+---------+
(player_id, event_date) is the primary key of this table.
This table shows the activity of players of some game.
Each row is a record of a player who logged in and played a number of games (possibly 0) before logging out on some day using some device.
 

Write an SQL query that reports for each player and date, how many games played so far by the player. That is, the total number of games played by the player until that date. Check the example for clarity.

The query result format is in the following example:

Activity table:
+-----------+-----------+------------+--------------+
| player_id | device_id | event_date | games_played |
+-----------+-----------+------------+--------------+
| 1         | 2         | 2016-03-01 | 5            |
| 1         | 2         | 2016-05-02 | 6            |
| 1         | 3         | 2017-06-25 | 1            |
| 3         | 1         | 2016-03-02 | 0            |
| 3         | 4         | 2018-07-03 | 5            |
+-----------+-----------+------------+--------------+

Result table:
+-----------+------------+---------------------+
| player_id | event_date | games_played_so_far |
+-----------+------------+---------------------+
| 1         | 2016-03-01 | 5                   |
| 1         | 2016-05-02 | 11                  |
| 1         | 2017-06-25 | 12                  |
| 3         | 2016-03-02 | 0                   |
| 3         | 2018-07-03 | 5                   |
+-----------+------------+---------------------+
For the player with id 1, 5 + 6 = 11 games played by 2016-05-02, and 5 + 6 + 1 = 12 games played by 2017-06-25.
For the player with id 3, 0 + 5 = 5 games played by 2018-07-03.
Note that for each player we only care about the days when the player logged in.

```

**--- SOLUTION ---**

```
SELECT
    player_id,
    event_date,
    SUM(games_played) OVER(PARTITION BY player_id ORDER BY event_date) AS games_played_so_far
FROM activity
```
***------------------------------------------------------------------------------------------------------------------------------------------------------------------------***


##  Project Employees II

[10] https://leetcode.com/problems/project-employees-ii/

Table: Project
```
+-------------+---------+
| Column Name | Type    |
+-------------+---------+
| project_id  | int     |
| employee_id | int     |
+-------------+---------+
(project_id, employee_id) is the primary key of this table.
employee_id is a foreign key to Employee table.
 

Table: Employee

+------------------+---------+
| Column Name      | Type    |
+------------------+---------+
| employee_id      | int     |
| name             | varchar |
| experience_years | int     |
+------------------+---------+
employee_id is the primary key of this table.
 

Write an SQL query that reports all the projects that have the most employees.
The query result format is in the following example:

Project table:
+-------------+-------------+
| project_id  | employee_id |
+-------------+-------------+
| 1           | 1           |
| 1           | 2           |
| 1           | 3           |
| 2           | 1           |
| 2           | 4           |
+-------------+-------------+

Employee table:
+-------------+--------+------------------+
| employee_id | name   | experience_years |
+-------------+--------+------------------+
| 1           | Khaled | 3                |
| 2           | Ali    | 2                |
| 3           | John   | 1                |
| 4           | Doe    | 2                |
+-------------+--------+------------------+

Result table:
+-------------+
| project_id  |
+-------------+
| 1           |
+-------------+
The first project has 3 employees while the second one has 2.
```

**--- SOULTION ---**
```
SELECT 
    project_id
FROM project
GROUP BY 
    project_id
HAVING 
    COUNT(employee_id) = (
    SELECT 
        COUNT(employee_id)
    FROM project
    GROUP BY 
        project_id
    ORDER BY 
        COUNT(employee_id) desc
    LIMIT 1
)
```

**-- USING WINDOW FUNCTION --**
```
SELECT 
    project_id 
FROM (
    SELECT
        project_id,
        COUNT(employee_id) AS total_emp,
        RANK() OVER(ORDER BY COUNT(employee_id) DESC) AS rnk
    FROM Project
    GROUP BY
        project_id
) AS stg
WHERE 
    rnk =1
```


***------------------------------------------------------------------------------------------------------------------------------------------------------------------------***

## Active Businesses

[11] https://leetcode.com/problems/active-businesses/
    
Table: Events

```
+---------------+---------+
| Column Name   | Type    |
+---------------+---------+
| business_id   | int     |
| event_type    | varchar |
| occurences    | int     | 
+---------------+---------+

```
(business_id, event_type) is the primary key of this table.
Each row in the table logs the info that an event of some type occured at some business for a number of times.
 

Write an SQL query to find all active businesses.

An active business is a business that has more than one event type with occurences greater than the average occurences of that event type among all businesses.

The query result format is in the following example:

```
Events table:
+-------------+------------+------------+
| business_id | event_type | occurences |
+-------------+------------+------------+
| 1           | reviews    | 7          |
| 3           | reviews    | 3          |
| 1           | ads        | 11         |
| 2           | ads        | 7          |
| 3           | ads        | 6          |
| 1           | page views | 3          |
| 2           | page views | 12         |
+-------------+------------+------------+

Result table:
+-------------+
| business_id |
+-------------+
| 1           |
+-------------+ 
Average for 'reviews', 'ads' and 'page views' are (7+3)/2=5, (11+7+6)/3=8, (3+12)/2=7.5 respectively.
Business with id 1 has 7 'reviews' events (more than 5) and 11 'ads' events (more than 8) so it is an active business.
```

**-- SOLUTION --**

```
WITH event_stg AS (
SELECT
    business_id,
    event_type,
    occurences,
    AVG(occurences) OVER(PARTITION by event_type) avg_occurence
FROM Events
)
SELECT 
    business_id
FROM event_stg
WHERE 
    occurences > avg_occurence
GROUP BY 
    business_id
HAVING 
    COUNT(DISTINCT event_type)>1
```
***------------------------------------------------------------------------------------------------------------------------------------------------------------------------***

## Unpopular Books

[12] https://leetcode.com/problems/unpopular-books/submissions/

Table: Books

```
+----------------+---------+
| Column Name    | Type    |
+----------------+---------+
| book_id        | int     |
| name           | varchar |
| available_from | date    |
+----------------+---------+
book_id is the primary key of this table.
Table: Orders

+----------------+---------+
| Column Name    | Type    |
+----------------+---------+
| order_id       | int     |
| book_id        | int     |
| quantity       | int     |
| dispatch_date  | date    |
+----------------+---------+
order_id is the primary key of this table.
book_id is a foreign key to the Books table.

Write an SQL query that reports the books that have sold less than 10 copies in the last year, excluding books that have been available for less than 1 month from today. Assume today is 2019-06-23.

The query result format is in the following example:

Books table:
+---------+--------------------+----------------+
| book_id | name               | available_from |
+---------+--------------------+----------------+
| 1       | "Kalila And Demna" | 2010-01-01     |
| 2       | "28 Letters"       | 2012-05-12     |
| 3       | "The Hobbit"       | 2019-06-10     |
| 4       | "13 Reasons Why"   | 2019-06-01     |
| 5       | "The Hunger Games" | 2008-09-21     |
+---------+--------------------+----------------+

Orders table:
+----------+---------+----------+---------------+
| order_id | book_id | quantity | dispatch_date |
+----------+---------+----------+---------------+
| 1        | 1       | 2        | 2018-07-26    |
| 2        | 1       | 1        | 2018-11-05    |
| 3        | 3       | 8        | 2019-06-11    |
| 4        | 4       | 6        | 2019-06-05    |
| 5        | 4       | 5        | 2019-06-20    |
| 6        | 5       | 9        | 2009-02-02    |
| 7        | 5       | 8        | 2010-04-13    |
+----------+---------+----------+---------------+

Result table:
+-----------+--------------------+
| book_id   | name               |
+-----------+--------------------+
| 1         | "Kalila And Demna" |
| 2         | "28 Letters"       |
| 5         | "The Hunger Games" |
+-----------+--------------------+
```
 
 **-- SOLUTION USING JOIN --**
 ```
 WITH copies_sold AS (
    SELECT
        book_id,
        SUM(quantity) AS sold_cnt
    FROM Orders
    WHERE dispatch_date BETWEEN DATE_ADD('2019-06-23', INTERVAL -1 YEAR) and DATE('2019-06-23') 
    GROUP BY 
        book_id
),
books AS (
SELECT
    book_id,
    name
FROM Books
WHERE available_from < DATE_ADD('2019-06-23', INTERVAL -1 MONTH) 
)
SELECT 
    books.book_id,
    name
FROM books 
LEFT JOIN copies_sold
ON copies_sold.book_id = books.book_id
WHERE 
    COALESCE(sold_cnt,0) < 10
```

**-- SOLUTION USING SUB QUERY --**
```
WITH temp AS (
SELECT
    book_id
FROM Orders
WHERE dispatch_date BETWEEN '2018-06-24' AND '2019-06-23'
GROUP BY 1
HAVING 
    SUM(quantity)>=10
)
SELECT
    book_id, 
    name
FROM Books
WHERE 
    available_from < '2019-05-24' 
    AND book_id NOT IN (
        SELECT 
            book_id 
        FROM temp
    )
```
***------------------------------------------------------------------------------------------------------------------------------------------------------------------------***

## Customers Who Bought All Products

[13] https://leetcode.com/problems/customers-who-bought-all-products/

Table: Customer
```
+-------------+---------+
| Column Name | Type    |
+-------------+---------+
| customer_id | int     |
| product_key | int     |
+-------------+---------+
product_key is a foreign key to Product table.
 

Table: Product

+-------------+---------+
| Column Name | Type    |
+-------------+---------+
| product_key | int     |
+-------------+---------+
product_key is the primary key column for this table.
 

Write an SQL query for a report that provides the customer ids from the Customer table that bought all the products in the Product table.

Return the result table in any order.

The query result format is in the following example:

Customer table:
+-------------+-------------+
| customer_id | product_key |
+-------------+-------------+
| 1           | 5           |
| 2           | 6           |
| 3           | 5           |
| 3           | 6           |
| 1           | 6           |
+-------------+-------------+

Product table:
+-------------+
| product_key |
+-------------+
| 5           |
| 6           |
+-------------+

Result table:
+-------------+
| customer_id |
+-------------+
| 1           |
| 3           |
+-------------+
The customers who bought all the products (5 and 6) are customers with id 1 and 3.
```

**-- SOLUTION --**
```
SELECT
    customer_id
FROM Customer
GROUP BY
    customer_id
HAVING 
    COUNT(DISTINCT product_key) = (
        SELECT 
            COUNT(DISTINCT product_key)
        FROM Product
    )
```
***------------------------------------------------------------------------------------------------------------------------------------------------------------------------***

## Swap salaries

[14] https://leetcode.com/problems/swap-salary/

Table: Salary
```
+-------------+----------+
| Column Name | Type     |
+-------------+----------+
| id          | int      |
| name        | varchar  |
| sex         | ENUM     |
| salary      | int      |
+-------------+----------+
id is the primary key for this table.
The sex column is ENUM value of type ('m', 'f').
The table contains information about an employee.
 

Write an SQL query to swap all 'f' and 'm' values (i.e., change all 'f' values to 'm' and vice versa) with a single update statement and no intermediate temp table(s).

Note that you must write a single update statement, DO NOT write any select statement for this problem.

The query result format is in the following example:

Salary table:
+----+------+-----+--------+
| id | name | sex | salary |
+----+------+-----+--------+
| 1  | A    | m   | 2500   |
| 2  | B    | f   | 1500   |
| 3  | C    | m   | 5500   |
| 4  | D    | f   | 500    |
+----+------+-----+--------+

Result table:
+----+------+-----+--------+
| id | name | sex | salary |
+----+------+-----+--------+
| 1  | A    | f   | 2500   |
| 2  | B    | m   | 1500   |
| 3  | C    | f   | 5500   |
| 4  | D    | m   | 500    |
+----+------+-----+--------+
(1, A) and (3, C) were changed from 'm' to 'f'.
(2, B) and (4, D) were changed from 'f' to 'm'.

```

**-- SOLUTION --**
```
UPDATE Salary
SET sex=(
    CASE WHEN sex='m' THEN 'f'
         WHEN sex='f' THEN 'm'
    END    
)
```
***------------------------------------------------------------------------------------------------------------------------------------------------------------------------***

## User Activity for the Past 30 Days II 

PAY ATTENTION TO THE QUESTION

[15] https://leetcode.com/problems/user-activity-for-the-past-30-days-ii/
    
```
Table: Activity

+---------------+---------+
| Column Name   | Type    |
+---------------+---------+
| user_id       | int     |
| session_id    | int     |
| activity_date | date    |
| activity_type | enum    |
+---------------+---------+
There is no primary key for this table, it may have duplicate rows.
The activity_type column is an ENUM of type ('open_session', 'end_session', 'scroll_down', 'send_message').
The table shows the user activities for a social media website.
Note that each session belongs to exactly one user.
 

Write an SQL query to find the average number of sessions per user for a period of 30 days ending 2019-07-27 inclusively, rounded to 2 decimal places. The sessions we want to count for a user are those with at least one activity in that time period.

The query result format is in the following example:

Activity table:
+---------+------------+---------------+---------------+
| user_id | session_id | activity_date | activity_type |
+---------+------------+---------------+---------------+
| 1       | 1          | 2019-07-20    | open_session  |
| 1       | 1          | 2019-07-20    | scroll_down   |
| 1       | 1          | 2019-07-20    | end_session   |
| 2       | 4          | 2019-07-20    | open_session  |
| 2       | 4          | 2019-07-21    | send_message  |
| 2       | 4          | 2019-07-21    | end_session   |
| 3       | 2          | 2019-07-21    | open_session  |
| 3       | 2          | 2019-07-21    | send_message  |
| 3       | 2          | 2019-07-21    | end_session   |
| 3       | 5          | 2019-07-21    | open_session  |
| 3       | 5          | 2019-07-21    | scroll_down   |
| 3       | 5          | 2019-07-21    | end_session   |
| 4       | 3          | 2019-06-25    | open_session  |
| 4       | 3          | 2019-06-25    | end_session   |
+---------+------------+---------------+---------------+

Result table:
+---------------------------+ 
| average_sessions_per_user |
+---------------------------+ 
| 1.33                      |
+---------------------------+ 
User 1 and 2 each had 1 session in the past 30 days while user 3 had 2 sessions so the average is (1 + 1 + 2) / 3 = 1.33.
```

**-- SOLUTION --**
```
SELECT
    ROUND(
        COALESCE(
            COUNT(DISTINCT session_id)/ 
            COUNT(DISTINCT user_id)
            , 0.00
        ), 2
    ) AS average_sessions_per_user
FROM Activity
WHERE activity_date 
    BETWEEN DATE_ADD('2019-07-27', INTERVAL -29 DAY) AND DATE('2019-07-27')
    
```
***------------------------------------------------------------------------------------------------------------------------------------------------------------------------***

## Game Play Analysis IV

[16] https://leetcode.com/problems/game-play-analysis-iv/

Table: Activity

```
+--------------+---------+
| Column Name  | Type    |
+--------------+---------+
| player_id    | int     |
| device_id    | int     |
| event_date   | date    |
| games_played | int     |
+--------------+---------+
(player_id, event_date) is the primary key of this table.
This table shows the activity of players of some game.
Each row is a record of a player who logged in and played a number of games (possibly 0) before logging out on some day using some device.
 

Write an SQL query that reports the fraction of players that logged in again on the day after the day they first logged in, rounded to 2 decimal places. In other words, you need to count the number of players that logged in for at least two consecutive days starting from their first login date, then divide that number by the total number of players.

The query result format is in the following example:

Activity table:
+-----------+-----------+------------+--------------+
| player_id | device_id | event_date | games_played |
+-----------+-----------+------------+--------------+
| 1         | 2         | 2016-03-01 | 5            |
| 1         | 2         | 2016-03-02 | 6            |
| 2         | 3         | 2017-06-25 | 1            |
| 3         | 1         | 2016-03-02 | 0            |
| 3         | 4         | 2018-07-03 | 5            |
+-----------+-----------+------------+--------------+

Result table:
+-----------+
| fraction  |
+-----------+
| 0.33      |
+-----------+
Only the player with id 1 logged back in after the first day he had logged in so the answer is 1/3 = 0.33
```

**-- SOLUTION -- using MIN AND LEAD IN CTE**

```
WITH next_days AS (
SELECT
    player_id,
    event_date,
    LEAD(event_date) OVER(PARTITION BY player_id ORDER BY event_date) AS next_date,
    MIN(event_date) OVER(PARTITION BY player_id) AS min_date
FROM Activity
)
SELECT 
    # COUNT(DISTINCT IF(DATEDIFF(next_date, event_date) = 1,player_id, NULL)),
    # COUNT(DISTINCT player_id),
    ROUND(
        COUNT(DISTINCT IF(DATEDIFF(next_date, event_date) = 1 AND min_date = event_date
                          ,player_id
                          ,NULL))/
        COUNT(DISTINCT player_id)
        ,2
    )AS fraction
FROM next_days
```

**-- SOLUTION -- using only MIN in CTE**

```
WITH first_two_login AS (
SELECT
    player_id,
    device_id, 
    event_date,
    MIN(event_date) OVER(PARTITION BY player_id) as first_login
FROM Activity
)
SELECT 
 ROUND(
     COUNT(DISTINCT player_id)/
        (SELECT COUNT(DISTINCT player_id) FROM Activity)
     ,2) AS fraction
FROM first_two_login
WHERE DATEDIFF(event_date,first_login) =1
```

**-- SOLUTION -- using self join **
```
WITH player_min_date AS (
SELECT
    player_id,
    MIN(event_date) AS first_start_date
FROM Activity 
GROUP BY 
    player_id
),
stg AS(
SELECT
    m.player_id,
    first_start_date,
    event_date AS next_event_date
FROM player_min_date m
LEFT JOIN Activity a
ON a.player_id = m.player_id
AND first_start_date = event_date - INTERVAL '1' DAY
)
SELECT
    ROUND(
        COUNT(DISTINCT IF (next_event_date IS NOT NULL, player_id, NULL))/
        COUNT(DISTINCT player_id)
        ,2) AS fraction
FROM stg
```

***------------------------------------------------------------------------------------------------------------------------------------------------------------------------***

## Last Person to Fit in the Elevator

[17] https://leetcode.com/problems/last-person-to-fit-in-the-elevator/

```
Table: Queue

+-------------+---------+
| Column Name | Type    |
+-------------+---------+
| person_id   | int     |
| person_name | varchar |
| weight      | int     |
| turn        | int     |
+-------------+---------+
person_id is the primary key column for this table.
This table has the information about all people waiting for an elevator.
The person_id and turn columns will contain all numbers from 1 to n, where n is the number of rows in the table.
 

The maximum weight the elevator can hold is 1000.

Write an SQL query to find the person_name of the last person who will fit in the elevator without exceeding the weight limit. It is guaranteed that the person who is first in the queue can fit in the elevator.

The query result format is in the following example:

Queue table
+-----------+-------------------+--------+------+
| person_id | person_name       | weight | turn |
+-----------+-------------------+--------+------+
| 5         | George Washington | 250    | 1    |
| 3         | John Adams        | 350    | 2    |
| 6         | Thomas Jefferson  | 400    | 3    |
| 2         | Will Johnliams    | 200    | 4    |
| 4         | Thomas Jefferson  | 175    | 5    |
| 1         | James Elephant    | 500    | 6    |
+-----------+-------------------+--------+------+

Result table
+-------------------+
| person_name       |
+-------------------+
| Thomas Jefferson  |
+-------------------+

Queue table is ordered by turn in the example for simplicity.
In the example George Washington(id 5), John Adams(id 3) and Thomas Jefferson(id 6) will enter the elevator as their weight sum is 250 + 350 + 400 = 1000.
Thomas Jefferson(id 6) is the last person to fit in the elevator because he has the last turn in these three people.
```

**-- SOLUTION USING SUBQUERY --**
```
WITH stg AS (
SELECT
    person_name,
    SUM(weight) OVER(ORDER BY turn) As rolling_weight,
    turn
FROM Queue
)
SELECT 
    person_name
FROM stg
WHERE turn = (
SELECT
    MAX(turn)
FROM stg
WHERE rolling_weight <=1000
)
```

**-- SOLUTION USING ORDER BY --**
```
WITH stg AS (
SELECT
    person_name,
    SUM(weight) OVER(ORDER BY turn) As rolling_weight,
    turn
FROM Queue
)
SELECT 
    person_name
FROM stg
WHERE
    rolling_weight <=1000
ORDER BY turn DESC
LIMIT 1
```
***------------------------------------------------------------------------------------------------------------------------------------------------------------------------***

## Invalid Tweets

[18] https://leetcode.com/problems/invalid-tweets/

Table: Tweets

```
+----------------+---------+
| Column Name    | Type    |
+----------------+---------+
| tweet_id       | int     |
| content        | varchar |
+----------------+---------+
tweet_id is the primary key for this table.
This table contains all the tweets in a social media app.
 

Write an SQL query to find the IDs of the invalid tweets. The tweet is invalid if the number of characters used in the content of the tweet is strictly greater than 15.

Return the result table in any order.

The query result format is in the following example:

Tweets table:
+----------+----------------------------------+
| tweet_id | content                          |
+----------+----------------------------------+
| 1        | Vote for Biden                   |
| 2        | Let us make America great again! |
+----------+----------------------------------+

Result table:
+----------+
| tweet_id |
+----------+
| 2        |
+----------+
Tweet 1 has length = 14. It is a valid tweet.
Tweet 2 has length = 32. It is an invalid tweet.
```

**-- SOLUTION --**

```
SELECT
    tweet_id
FROM tweets
WHERE LENGTH(content)>15
```
***------------------------------------------------------------------------------------------------------------------------------------------------------------------------***

## Running Total for Different Genders

[19] https://leetcode.com/problems/running-total-for-different-genders/

```
Table: Scores

+---------------+---------+
| Column Name   | Type    |
+---------------+---------+
| player_name   | varchar |
| gender        | varchar |
| day           | date    |
| score_points  | int     |
+---------------+---------+
(gender, day) is the primary key for this table.
A competition is held between females team and males team.
Each row of this table indicates that a player_name and with gender has scored score_point in someday.
Gender is 'F' if the player is in females team and 'M' if the player is in males team.
 

Write an SQL query to find the total score for each gender at each day.

Order the result table by gender and day

The query result format is in the following example:

Scores table:
+-------------+--------+------------+--------------+
| player_name | gender | day        | score_points |
+-------------+--------+------------+--------------+
| Aron        | F      | 2020-01-01 | 17           |
| Alice       | F      | 2020-01-07 | 23           |
| Bajrang     | M      | 2020-01-07 | 7            |
| Khali       | M      | 2019-12-25 | 11           |
| Slaman      | M      | 2019-12-30 | 13           |
| Joe         | M      | 2019-12-31 | 3            |
| Jose        | M      | 2019-12-18 | 2            |
| Priya       | F      | 2019-12-31 | 23           |
| Priyanka    | F      | 2019-12-30 | 17           |
+-------------+--------+------------+--------------+
Result table:
+--------+------------+-------+
| gender | day        | total |
+--------+------------+-------+
| F      | 2019-12-30 | 17    |
| F      | 2019-12-31 | 40    |
| F      | 2020-01-01 | 57    |
| F      | 2020-01-07 | 80    |
| M      | 2019-12-18 | 2     |
| M      | 2019-12-25 | 13    |
| M      | 2019-12-30 | 26    |
| M      | 2019-12-31 | 29    |
| M      | 2020-01-07 | 36    |
+--------+------------+-------+
For females team:
First day is 2019-12-30, Priyanka scored 17 points and the total score for the team is 17.
Second day is 2019-12-31, Priya scored 23 points and the total score for the team is 40.
Third day is 2020-01-01, Aron scored 17 points and the total score for the team is 57.
Fourth day is 2020-01-07, Alice scored 23 points and the total score for the team is 80.
For males team:
First day is 2019-12-18, Jose scored 2 points and the total score for the team is 2.
Second day is 2019-12-25, Khali scored 11 points and the total score for the team is 13.
Third day is 2019-12-30, Slaman scored 13 points and the total score for the team is 26.
Fourth day is 2019-12-31, Joe scored 3 points and the total score for the team is 29.
Fifth day is 2020-01-07, Bajrang scored 7 points and the total score for the team is 36.
```

**-- SOLUTION --**
```
SELECT 
    gender, 
    day, 
    SUM(score_points) OVER (PARTITION BY gender ORDER BY day) AS total 
FROM scores
```
***------------------------------------------------------------------------------------------------------------------------------------------------------------------------***

## Customer Order Frequency

[20] https://leetcode.com/problems/customer-order-frequency/

Table: Customers
```
+---------------+---------+
| Column Name   | Type    |
+---------------+---------+
| customer_id   | int     |
| name          | varchar |
| country       | varchar |
+---------------+---------+
customer_id is the primary key for this table.
This table contains information of the customers in the company.
 

Table: Product

+---------------+---------+
| Column Name   | Type    |
+---------------+---------+
| product_id    | int     |
| description   | varchar |
| price         | int     |
+---------------+---------+
product_id is the primary key for this table.
This table contains information of the products in the company.
price is the product cost.

Table: Orders

+---------------+---------+
| Column Name   | Type    |
+---------------+---------+
| order_id      | int     |
| customer_id   | int     |
| product_id    | int     |
| order_date    | date    |
| quantity      | int     |
+---------------+---------+
order_id is the primary key for this table.
This table contains information on customer orders.
customer_id is the id of the customer who bought "quantity" products with id "product_id".
Order_date is the date in format ('YYYY-MM-DD') when the order was shipped.
 

Write an SQL query to report the customer_id and customer_name of customers who have spent at least 100 dollar in each month of June and July 2020.

Return the result table in any order.

The query result format is in the following example.


Customers
+--------------+-----------+-------------+
| customer_id  | name      | country     |
+--------------+-----------+-------------+
| 1            | Winston   | USA         |
| 2            | Jonathan  | Peru        |
| 3            | Moustafa  | Egypt       |
+--------------+-----------+-------------+

Product
+--------------+-------------+-------------+
| product_id   | description | price       |
+--------------+-------------+-------------+
| 10           | LC Phone    | 300         |
| 20           | LC T-Shirt  | 10          |
| 30           | LC Book     | 45          |
| 40           | LC Keychain | 2           |
+--------------+-------------+-------------+

Orders
+--------------+-------------+-------------+-------------+-----------+
| order_id     | customer_id | product_id  | order_date  | quantity  |
+--------------+-------------+-------------+-------------+-----------+
| 1            | 1           | 10          | 2020-06-10  | 1         |
| 2            | 1           | 20          | 2020-07-01  | 1         |
| 3            | 1           | 30          | 2020-07-08  | 2         |
| 4            | 2           | 10          | 2020-06-15  | 2         |
| 5            | 2           | 40          | 2020-07-01  | 10        |
| 6            | 3           | 20          | 2020-06-24  | 2         |
| 7            | 3           | 30          | 2020-06-25  | 2         |
| 9            | 3           | 30          | 2020-05-08  | 3         |
+--------------+-------------+-------------+-------------+-----------+

Result table:
+--------------+------------+
| customer_id  | name       |  
+--------------+------------+
| 1            | Winston    |
+--------------+------------+ 
```
Winston spent 300 dollar (300 * 1) in June and 100 ( 10 * 1 + 45 * 2) in July 2020.
Jonathan spent 600 dollar (300 * 2) in June and 20 ( 2 * 10) in July 2020.
Moustafa spent 110 dollar (10 * 2 + 45 * 2) in June and 0 in July 2020.

**-- SOLUTION --**
```
WITH stg AS (
SELECT
    c.customer_id,
    name,
    SUM(IF(DATE_FORMAT(order_date, '%Y-%m')='2020-06', quantity*price,0)) AS june_sum,
    SUM(IF(DATE_FORMAT(order_date, '%Y-%m')='2020-07', quantity*price,0)) AS july_sum
FROM Customers c
JOIN Orders o
ON o.customer_id = c.customer_id
JOIN Product p
ON p.product_id = o.product_id
WHERE DATE_FORMAT(order_date, '%Y-%m') IN ('2020-06','2020-07') 
GROUP BY 
    1,2
)
SELECT
    customer_id,
    name
FROM stg
WHERE 
    june_sum >= 100 
    AND july_sum >=100
```

**-- worstu solution --**
```
WITH order_stg AS (
SELECT
    order_id,
    customer_id,
    product_id,
    DATE_FORMAT(order_date, '%Y-%m') AS order_mnt,
    quantity
FROM Orders
WHERE 
    DATE_FORMAT(order_date, '%Y-%m') IN ('2020-06', '2020-07')
)
SELECT
    customer_id,
    name
FROM(
    SELECT
        order_mnt,
        order_stg.customer_id,
        name,
        SUM(quantity * price) AS total
    FROM order_stg
    JOIN Product
    ON order_stg.product_id = Product.product_id
    JOIN Customers
    ON order_stg.customer_id = Customers.customer_id
    GROUP BY
        1,2,3
) AS stg
WHERE total>=100
GROUP BY
    customer_id,
    name
HAVING
    COUNT(1)=2
    ```
***------------------------------------------------------------------------------------------------------------------------------------------------------------------------***

## Managers with at Least 5 Direct Reports

[21]  https://leetcode.com/problems/managers-with-at-least-5-direct-reports/

The Employee table holds all employees including their managers. Every employee has an Id, and there is also a column for the manager Id.
```
+------+----------+-----------+----------+
|Id    |Name 	  |Department |ManagerId |
+------+----------+-----------+----------+
|101   |John 	  |A 	      |null      |
|102   |Dan 	  |A 	      |101       |
|103   |James 	  |A 	      |101       |
|104   |Amy 	  |A 	      |101       |
|105   |Anne 	  |A 	      |101       |
|106   |Ron 	  |B 	      |101       |
+------+----------+-----------+----------+
Given the Employee table, write a SQL query that finds out managers with at least 5 direct report. For the above table, your SQL query should return:

+-------+
| Name  |
+-------+
| John  |
+-------+
```
**--SOLUTION--**
```
SELECT
    m.Name
FROM Employee m 
JOIN Employee e
ON m.Id = e.ManagerID
GROUP BY
    m.Name
HAVING 
    COUNT(e.Id) >=5
 
```
***------------------------------------------------------------------------------------------------------------------------------------------------------------------------***

##  Exchange Seats

[22] https://leetcode.com/problems/exchange-seats/

Mary is a teacher in a middle school and she has a table seat storing students' names and their corresponding seat ids.

The column id is continuous increment.

Mary wants to change seats for the adjacent students.

Can you write a SQL query to output the result for Mary?

**--SOLUTION--**
```
WITH stg AS (
SELECT  
    id,
    student,
    LEAD(student) OVER(ORDER BY id) next_student,
    LAG(student) OVER(ORDER BY id) prev_student
FROM seat
)
SELECT 
    id,
    COALESCE(IF(id%2=0, prev_student, next_student), student) AS student
FROM stg
 
```
***------------------------------------------------------------------------------------------------------------------------------------------------------------------------***

## Monthly Transactions II

[23] https://leetcode.com/problems/monthly-transactions-ii/submissions/

**--SOLUTION--**

```
WITH stg AS (
    SELECT
        id,
        country,
        state,
        amount,
        trans_date
    FROM Transactions

    UNION ALL

    SELECT
        trans_id AS id,
        country,
        'chargeback' AS state,
        amount,
        Chargebacks.trans_date
    FROM Chargebacks
    JOIN Transactions
    ON Transactions.id = Chargebacks.trans_id
)
SELECT 
    DATE_FORMAT(trans_date, '%Y-%m') AS month,
    country,
    COUNT(IF(state='approved', 1, NULL)) AS approved_count,
    SUM(IF(state='approved', amount, 0)) AS approved_amount,
    COUNT(IF(state='chargeback', 1, NULL)) AS chargeback_count,
    SUM(IF(state='chargeback', amount, 0)) AS chargeback_amount
FROM stg
GROUP BY 
 1, 2
 HAVING 
    approved_count + approved_amount + chargeback_count + chargeback_amount > 0
```
***------------------------------------------------------------------------------------------------------------------------------------------------------------------------***

## Page Recommendations

[24] https://leetcode.com/problems/page-recommendations/

**--SOLUTION--**
```
WITH user_stg AS(
SELECT
    user1_id AS user_A,
    user2_id AS user_B
FROM Friendship  
WHERE 
    user1_id=1

UNION ALL

SELECT
    user2_id AS user_A,
    user1_id AS user_B
FROM Friendship
WHERE
    user2_id=1
)
SELECT DISTINCT
    page_id AS recommended_page
FROM Likes
WHERE
    user_id IN (
        SELECT
            user_B
        FROM user_stg
    )
    AND page_id NOT IN (
        SELECT
            page_id
        FROM Likes
        WHERE
            user_id=1
    )
```
***------------------------------------------------------------------------------------------------------------------------------------------------------------------------***

## Restaurant Growth
[25] https://leetcode.com/problems/restaurant-growth/


You are the restaurant owner and you want to analyze a possible expansion (there will be at least one customer every day).

Write an SQL query to compute moving average of how much customer paid in a 7 days window (current day + 6 days before) .

The query result format is in the following example:

Return result table ordered by visited_on.

average_amount should be rounded to 2 decimal places, all dates are in the format ('YYYY-MM-DD').


**--SOLUTION--**

```
WITH stg AS (
SELECT
    visited_on,
    SUM(amount) AS total_amount
FROM Customer
GROUP BY
    visited_on
)
SELECT
    visited_on,
    SUM(total_amount) OVER(ORDER BY visited_on ROWS BETWEEN 6 PRECEDING AND CURRENT ROW) AS amount,
    ROUND(
        AVG(total_amount)
          OVER(ORDER BY visited_on ROWS BETWEEN 6 PRECEDING AND CURRENT ROW)
        ,2) AS average_amount
FROM stg
LIMIT 9999 OFFSET 6
```

***------------------------------------------------------------------------------------------------------------------------------------------------------------------------***

## Activity Participants

[26] https://leetcode.com/problems/activity-participants/

Write an SQL query to find the names of all the activities with neither maximum, nor minimum number of participants.

Return the result table in any order. Each activity in table Activities is performed by any person in the table Friends.

**--SOLUTION--**
```
WITH stg AS (
SELECT
    activity,
    COUNT(id) AS total
FROM Friends
GROUP BY 
    activity
)
SELECT
    activity
FROM stg
WHERE
    total <> (SELECT MAX(total) FROM stg)
    AND total <> (SELECT MIN(total) FROM stg)
    
```

***------------------------------------------------------------------------------------------------------------------------------------------------------------------------***

## Customers Who Bought Products A and B but Not C

[27] https://leetcode.com/problems/customers-who-bought-products-a-and-b-but-not-c/

Write an SQL query to report the customer_id and customer_name of customers who bought products "A", "B" but did not buy the product "C" since we want to recommend them buy this product.

Return the result table ordered by customer_id.

The query result format is in the following example.

**--SOLUTION--**
```
WITH customer_order AS (
SELECT
    o.customer_id,
    customer_name,
    COUNT(IF(product_name='A',1,NULL)) AS bought_a,
    COUNT(IF(product_name='B',1,NULL)) AS bought_b
FROM Orders o
JOIN Customers c
ON c.customer_id = o.customer_id
GROUP BY
    1, 2
)
SELECT
    customer_id,
    customer_name
FROM customer_order
WHERE 
    bought_a > 0
    AND bought_b > 0
    AND customer_id NOT IN (
        
    SELECT
        customer_id
    FROM Orders
    WHERE 
        product_name = 'C'
    )
```

***------------------------------------------------------------------------------------------------------------------------------------------------------------------------***

## Evaluate Boolean Expression

[28] https://leetcode.com/problems/evaluate-boolean-expression/

Write an SQL query to evaluate the boolean expressions in Expressions table.

Return the result table in any order.

The query result format is in the following example.

**--SOLUTION--**
```
SELECT
    left_operand,
    # l.value AS left_value,
    operator,
    right_operand,
    # r.value AS right_value,
    CASE WHEN operator='>' THEN IF(l.value > r.value, 'true', 'false')
        WHEN operator= '=' THEN IF(l.value = r.value, 'true', 'false')
        WHEN operator= '<' THEN IF(l.value < r.value, 'true', 'false')
    END AS "value"
FROM Expressions e
JOIN Variables l
ON l.name = e.left_operand
JOIN Variables r
ON r.name = e.right_operand
```

***------------------------------------------------------------------------------------------------------------------------------------------------------------------------***

## Countries You Can Safely Invest In

[29] https://leetcode.com/problems/countries-you-can-safely-invest-in/

A telecommunications company wants to invest in new countries. The company intends to invest in the countries where the average call duration of the calls in this country is strictly greater than the global average call duration.

Write an SQL query to find the countries where this company can invest.

Return the result table in any order.

The query result format is in the following example:

**--SOLUTION--**
```
WITH total_calls AS (
    SELECT
        caller_id AS id,
        duration
    FROM Calls
    
    UNION ALL
    
    SELECT
        callee_id AS id,
        duration
    FROM Calls
),
stg AS (
SELECT
    c.name AS country,
    AVG(duration) AS avg_country,
    SUM(SUM(duration)) OVER() / SUM(COUNT(t.id)) OVER() AS avg_overall
FROM total_calls t
JOIN Person p
ON p.id = t.id
JOIN Country c
ON SUBSTR(p.phone_number,1,3) = c.country_code
GROUP BY 
    1
)
SELECT
    country
FROM stg
WHERE 
    avg_country> avg_overall
```
***------------------------------------------------------------------------------------------------------------------------------------------------------------------------***

## Find the Missing IDs

[30] https://leetcode.com/problems/find-the-missing-ids/

Write an SQL query to find the missing customer IDs. The missing IDs are ones that are not in the Customers table but are in the range between 1 and the maximum customer_id present in the table.

Notice that the maximum customer_id will not exceed 100.

Return the result table ordered by ids in ascending order.

The query result format is in the following example.

**--SOLUTION--**

```

WITH RECURSIVE cte (num) AS (
    SELECT
       1
    
    UNION ALL
    
    SELECT
        num+1
    FROM cte
    WHERE 
        num < (
        SELECT
            MAX(customer_id)
        FROM Customers
        )
)
SELECT 
    num AS ids
FROM cte
LEFT JOIN Customers
ON cte.num = Customers.customer_id
WHERE 
    customer_id IS NULL
ORDER BY
    ids
```
***------------------------------------------------------------------------------------------------------------------------------------------------------------------------***

## Number of Calls Between Two Persons

[31] https://leetcode.com/problems/number-of-calls-between-two-persons/submissions/

Write an SQL query to report the number of calls and the total call duration between each pair of distinct persons (person1, person2) where person1 < person2.

Return the result table in any order.

The query result format is in the following example:

**--SOLUTION--**
```
WITH person_call AS (
SELECT
    from_id AS person1,
    to_id AS person2,
    duration
FROM Calls
WHERE
    from_id < to_id
    
UNION ALL

SELECT
    to_id AS person1,
    from_id AS person2,
    duration
FROM Calls
WHERE
    from_id > to_id
)
SELECT
    person1,
    person2,
    COUNT(duration) AS call_count,
    SUM(duration) AS total_duration
FROM person_call
GROUP BY 
    1, 2
```
***------------------------------------------------------------------------------------------------------------------------------------------------------------------------***

# Hard Questions

## Department Top Three Salaries
https://leetcode.com/problems/department-top-three-salaries/


The Employee table holds all employees. Every employee has an Id, and there is also a column for the department Id.

```
+----+-------+--------+--------------+
| Id | Name  | Salary | DepartmentId |
+----+-------+--------+--------------+
| 1  | Joe   | 85000  | 1            |
| 2  | Henry | 80000  | 2            |
| 3  | Sam   | 60000  | 2            |
| 4  | Max   | 90000  | 1            |
| 5  | Janet | 69000  | 1            |
| 6  | Randy | 85000  | 1            |
| 7  | Will  | 70000  | 1            |
+----+-------+--------+--------------+
The Department table holds all departments of the company.

+----+----------+
| Id | Name     |
+----+----------+
| 1  | IT       |
| 2  | Sales    |
+----+----------+
Write a SQL query to find employees who earn the top three salaries in each of the department. For the above tables, your SQL query should return the following rows (order of rows does not matter).

+------------+----------+--------+
| Department | Employee | Salary |
+------------+----------+--------+
| IT         | Max      | 90000  |
| IT         | Randy    | 85000  |
| IT         | Joe      | 85000  |
| IT         | Will     | 70000  |
| Sales      | Henry    | 80000  |
| Sales      | Sam      | 60000  |
+------------+----------+--------+

```

**-- SOLUTION --**

```
WITH emp_rank AS (
    SELECT
        dpt.name AS department,
        emp.name AS employee,
        salary,
        DENSE_RANK() OVER(PARTITION BY dpt.name ORDER BY salary DESC) as rnk
    FROM Employee emp
    JOIN Department dpt
        ON emp.departmentid = dpt.id
)
SELECT 
    department,
    employee,
    salary
FROM emp_rank
WHERE 
    rnk <=3
 ```
 ***------------------------------------------------------------------------------------------------------------------------------------------------------------------------***

## Median Salary
https://leetcode.com/problems/median-employee-salary/

The Employee table holds all employees. The employee table has three columns: Employee Id, Company Name, and Salary.

```
+-----+------------+--------+
|Id   | Company    | Salary |
+-----+------------+--------+
|1    | A          | 2341   |
|2    | A          | 341    |
|3    | A          | 15     |
|4    | A          | 15314  |
|5    | A          | 451    |
|6    | A          | 513    |
|7    | B          | 15     |
|8    | B          | 13     |
|9    | B          | 1154   |
|10   | B          | 1345   |
|11   | B          | 1221   |
|12   | B          | 234    |
|13   | C          | 2345   |
|14   | C          | 2645   |
|15   | C          | 2645   |
|16   | C          | 2652   |
|17   | C          | 65     |
+-----+------------+--------+
Write a SQL query to find the median salary of each company. Bonus points if you can solve it without using any built-in SQL functions.

+-----+------------+--------+
|Id   | Company    | Salary |
+-----+------------+--------+
|5    | A          | 451    |
|6    | A          | 513    |
|12   | B          | 234    |
|9    | B          | 1154   |
|14   | C          | 2645   |
+-----+------------+--------+

```

**-- Solution --**

```
WITH stg AS (
    SELECT
        Id,
        Company,
        Salary,
        COUNT(salary) OVER(PARTITION BY Company) AS total_rows,
        ROW_NUMBER() OVER(PARTITION BY Company ORDER BY salary) AS row_num
    FROM Employee
)
SELECT
    Id,
    Company,
    Salary
FROM stg
WHERE 
    row_num BETWEEN total_rows/2  AND (total_rows/2) +1
```
***------------------------------------------------------------------------------------------------------------------------------------------------------------------------***

## Find Cumulative Salary of an Employee
https://leetcode.com/problems/find-cumulative-salary-of-an-employee/

The Employee table holds the salary information in a year.

Write a SQL to get the cumulative sum of an employee's salary over a period of 3 months but exclude the most recent month.

The result should be displayed by 'Id' ascending, and then by 'Month' descending.
```
Input

| Id | Month | Salary |
|----|-------|--------|
| 1  | 1     | 20     |
| 2  | 1     | 20     |
| 1  | 2     | 30     |
| 2  | 2     | 30     |
| 3  | 2     | 40     |
| 1  | 3     | 40     |
| 3  | 3     | 60     |
| 1  | 4     | 60     |
| 3  | 4     | 70     |
Output

| Id | Month | Salary |
|----|-------|--------|
| 1  | 3     | 90     |
| 1  | 2     | 50     |
| 1  | 1     | 20     |
| 2  | 1     | 20     |
| 3  | 3     | 100    |
| 3  | 2     | 40     |
```

**-- Solution --**
```
WITH Employee_stg AS (
    SELECT
        Id,
        Month,
        Salary,
        ROW_NUMBER() OVER(PARTITION BY Id ORDER BY MONTH DESC) AS row_num
    FROM Employee
)
SELECT
    Id,
    Month,
    SUM(Salary) OVER(PARTITION BY Id 
                        ORDER BY Month ROWS BETWEEN 2 PRECEDING AND CURRENT ROW) AS Salary
FROM Employee_stg
WHERE 
    row_num > 1
ORDER BY Id, Month DESC
```
***------------------------------------------------------------------------------------------------------------------------------------------------------------------------***

## Find Median Given Frequency of Numbers

https://leetcode.com/problems/find-median-given-frequency-of-numbers/

```
The Numbers table keeps the value of number and its frequency.

+----------+-------------+
|  Number  |  Frequency  |
+----------+-------------|
|  0       |  7          |
|  1       |  1          |
|  2       |  3          |
|  3       |  1          |
+----------+-------------+
In this table, the numbers are 0, 0, 0, 0, 0, 0, 0, 1, 2, 2, 2, 3, so the median is (0 + 0) / 2 = 0.

+--------+
| median |
+--------|
| 0.0000 |
+--------+
Write a query to find the median of all numbers and name the result as median.
```


**-- SOLUTION --** 
```
WITH stg AS (
SELECT 
    Number, 
    Frequency,
    SUM(Frequency) OVER (ORDER BY Number) AS cumulative_freq,
    SUM(Frequency) OVER () AS total_freq
FROM Numbers
)
SELECT 
    AVG(Number) AS median
FROM stg
-- mid value is between start of freq and end of the frequency
WHERE total_freq / 2.0 BETWEEN cumulative_freq - Frequency AND cumulative_freq
```